## 6-2. PCA를 이용한 차원 축소
'**PCA(Principal Component Analysis)**'는 우리말로는 '주성분 분석'이라고 하며 데이터의 분산을 최대한 보존하는 새로운 축을 찾아 변환함으로써 차원을 축소하고자 하는 방법이다. 통계와 선형대수에 익숙하다면 수학적으로 PCA 과정을 풀어 이해하는 것을 추천한다. 이를 위해서는 고유값, 고유벡터, 공분산행렬과 같은 선형대수의 개념을 알아야 하는데, 이 책에서는 가급적 직관적으로 설명하고 이해하고자 한다. 사람은 4차원 이상의 공간은 사실상 직관적으로 이해가 불가능하므로 2차원 정도에서 최대한 설명을 한다.

우선 가장 중요한 원칙은 차원을 줄이는 과정에서도 정보가 최대한 유지돼야 한다는 것이다. 즉 10개의 독립변수를 9개로 줄인다고 해도 원래 가지고 있던 정보량을 최대한 유지하려면 새로운 변수 9개가 원래 10개의 변수가 가지는 정보량을 최대한 반영해야 한다는 것이다. 통계적으로 이러한 정보량은 분산으로 측정할 수 있다. 차원이 줄어들 때 최대한 분산을 크게 유지한다면 정보량의 손실을 최소화할 수 있다.

2차원 공간에 점들이 분포할 때, 이 점들은 두 개의 변수(일반적인 좌표계에서 x, y)로 표현된다. 여기서 한 차원을 줄이면 일직선 위의 한 변수로 값들을 표현해야 한다. 2차원 공간의 점들을 3차원 공간에 놓고, 옆에서 2차원이 1차원이 되도록 바라본다고 생각해보면, 점들이 일직선상에 위치하는 것처럼 보일 것이며 바라보는 방향에 따라 점들이 분포하는 범위가 달리진다. 바라보는 방향 중 점들이 가장 넓게 분포하는 방향이 가장 좋은 선택이 되며 이때 분산도 가장 커진다. 점들이 넓게 분포하면 점이 겹쳐질 확률도 줄어들게 되고, 각 점 사이의 거리도 멀어져서 점을 더 잘 분별할 수 있을 것이다. 즉 정보량이 가장 많다고 할 수 있는 것이다. PCA는 이와 같이 분산을 가장 크게 만드는 새로운 축을 찾는 방법론이라고 할 수 있다.

PCA 실습을 위해 먼저 우리가 익숙한 20 뉴스그룹의 문서들을 아래와 같이 가져온다.

In [1]:
from sklearn.datasets import fetch_20newsgroups

# 20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# 학습 데이터셋을 가져옴
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'), # 메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류
                                      categories=categories)
# 검증 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test', 
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

앞에서 했던 방식으로 토큰화, 불용어처리, 스테밍 등의 전처리 이후 BOW 기반의 특성 벡터로 변환해 문서 분류를 위한 데이터를 준비한다.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

cachedStopWords = stopwords.words("english")

X_train = newsgroups_train.data # 학습 데이터셋 문서
y_train = newsgroups_train.target # 학습 데이터셋 라벨

X_test = newsgroups_test.data # 평가 데이터셋 문서
y_test = newsgroups_test.target # 평가 데이터셋 라벨

RegTok = RegexpTokenizer("[\w']{3,}") # 정규표현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english')) # 영어 불용어를 가져옴

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    # stopwords 제외
    words = [word for word in tokens if (word not in english_stops) and len(word) > 2]
    # porter stemmer 적용
    features = (list(map(lambda token: PorterStemmer().stem(token), words)))
    return features

tfidf = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf.fit_transform(X_train) # train set을 변환
X_test_tfidf = tfidf.transform(X_test) # test set을 변환

차원을 축소한 뒤의 분류 성능과 비교를 하기 위해, 사이킷런의 로지스틱 회귀분석 라이브러리를 이용해 아래와 같이 차원 축소 전 분류 성능을 미리 측정해 둔다.

In [3]:
from sklearn.linear_model import LogisticRegression

LR_clf = LogisticRegression() # 분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용해 분류기를 학습

print(f'# Train set score: {LR_clf.score(X_train_tfidf, y_train):.3f}') 
print(f'# Test set score: {LR_clf.score(X_test_tfidf, y_test):.3f}') 

# Train set score: 0.962
# Test set score: 0.761


사이킷런은 주성분 분석을 지원하기 위해 [PCA 라이브러리](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)를 제공한다. 매개변수 중에서 n_components는 축소하고자 하는 차원의 크기를 지정하며, svd_solver는 알고리즘을 선택할 수 있다. svd_solver의 경우 auto가 기본값인데, 축소 전 차원과 축소 목표 차원을 고려해 자동으로 선택하므로 특별히 신경 쓰지 않고 싶다면 기본값을 선택하면 된다.

객체의 속성값인 explained_variance_는 각각의 새로운 축이 설명하는 분산을 나타내고, explained_variance_ratio_는 이를 축소 전 분산에 대한 비율로 표시해준다. 만일 새로운 축들이 원래의 분산을 모두 설명해준다면 explained_variance_ratio_의 합은 1이 될 것이다.

다음 예시에서는 20,085였던 차원을 2,000차원으로 축소한다. 원래 차원은 아래와 같이 X_train_tfidf의 모양을 출력하면 알 수 있다. 사이킷런의 PCA는 희소 벡터 형식에 대해 직접적인 연산을 지원하지 않는다. 즉 CountVectorizer나 TfidfVectorizer로 변환한 행렬을 직접 인수로 사용할 수 없다. 따라서 아래와 같이 먼저 toarray() 메서드를 이용해 형태를 변환한 후, transform()이나 fit_transform() 메서드에 인수로 전달한다. 축소한 뒤 원래의 분산의 어느 정도를 설명하는지 보기 위해 explained_variance_ratio_의 합을 출력했다.

In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2000, random_state=7)
X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())

print('Original tfidf matrix shape:', X_train_tfidf.shape)
print('PCA Converted matrix shape:', X_train_pca.shape)
print(f'Sum of explained variance ratio: {pca.explained_variance_ratio_.sum():.3f}')

Original tfidf matrix shape: (2034, 20085)
PCA Converted matrix shape: (2034, 2000)
Sum of explained variance ratio: 1.000


결과를 보면 먼저 원래 차원이 20,085인데 이는 특성 벡터의 단어 수를 나타내고, 축소된 차원은 목표한 바와 같이 2,000이다. 원래 차원의 수가 커서 계산량이 상당히 많고 그로 인해 시간도 꽤 걸리는 것을 볼 수 있다. 시간은 걸렸으나, 차원이 거의 1/10로 줄었는데도 불구하고 설명되는 분산은 거의 100%이므로 정보의 손실이 거의 없는 것을 알 수 있다. 축소된 데이터를 이용해 문서 분류를 함으로써 성능의 변화를 살펴보자.

In [5]:
LR_clf.fit(X_train_pca, y_train)
print(f'# Train set score: {LR_clf.score(X_train_pca, y_train):.3f}')
print(f'# Test set score: {LR_clf.score(X_test_pca, y_test):.3f}')

# Train set score: 0.962
# Test set score: 0.761


위 결과를 보면, 축소하기 전 성능과 동일한 것을 확인할 수 있다. PCA는 선형결합을 통해 최대한의 정보를 유지하므로, 특성 선택과는 다른 성능을 낸다. 5장의 문서 분류에서 우리가 처음부터 특성의 수를 2,000개로 제한했을 때 테스트 집합헤 대한 성능이 0.734였던 것을 생각해보면 특성 추출이 더 좋은 성능을 보인다는 것을 확인할 수 있다.

그렇다면 라쏘 회귀분석을 이용한 특성 선택과의 비교는 어떨지 알아보기 위해, 먼저 라쏘 회귀분석을 아래와 같이 수행해본다.

In [6]:
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1)
lasso_clf.fit(X_train_tfidf, y_train)

print(f'# Train set score: {lasso_clf.score(X_train_tfidf, y_train):.3f}')
print(f'# Test set score: {lasso_clf.score(X_test_tfidf, y_test):.3f}')

# Train set score: 0.790
# Test set score: 0.718


In [7]:
import numpy as np

# 계수(coefficient) 중에서 0이 아닌 것들의 개수를 출력
print(f'# Used features count: {np.sum(lasso_clf.coef_ != 0)} out of', X_train_tfidf.shape[1]) 

# Used features count: 321 out of 20085


위 결과를 보면 최종적으로 사용된 특성은 321개이고, 그 결과 테스트 집합에 대한 성능은 0.718로 축소 전에 비해 꽤 많이 떨어지는 것을 볼 수 있다. PCA와의 비교를 위해, 아래와 같이 목표 차원을 라쏘 회귀분석과 동일한 321개로 설정한 후, 변환한 데이터로 학습을 해본다.

In [8]:
pca = PCA(n_components=321, random_state=7)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())
print('PCA Converted X shape:', X_train_pca.shape)
print(f'Sum of explained variance ratio: {pca.explained_variance_ratio_.sum():.3f}')

LR_clf.fit(X_train_pca, y_train)
print('# Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train)))
print('# Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test)))

PCA Converted X shape: (2034, 321)
Sum of explained variance ratio: 0.437
# Train set score: 0.875
# Test set score: 0.751


설명되는 분산의 비율이 43.7%로 꽤 많이 줄었음에도 불구하고 테스트 집합에 대한 정확도는 75.1%로 축소 전인 76.1%에 비해 겨우 1%만 줄었다. 따라서 라쏘 회귀분석과 축소된 차원은 동일하지만 분류기의 성능은 훨씬 뛰어난 것을 볼 수 있다. 이제 조금만 더 욕심을 내서 아래와 같이 차원을 100개로 줄여본다.

In [9]:
pca = PCA(n_components=100, random_state=7)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())
print('PCA Converted X shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))

LR_clf.fit(X_train_pca, y_train)
print('# Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train)))
print('# Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test)))

PCA Converted X shape: (2034, 100)
Sum of explained variance ratio: 0.211
# Train set score: 0.807
# Test set score: 0.738


테스트 집합에 대한 정확도가 73.8%로, 특성의 수가 321개인 위 라쏘 회귀분석보다 여전히 성능이 좋은 것을 볼 수 있다.

In [10]:
% store newsgroups_train
% store newsgroups_test
% store X_train
% store X_test
% store y_train
% store y_test
% store X_train_tfidf
% store X_test_tfidf
% store X_train_pca
% store X_test_pca

Stored 'newsgroups_train' (Bunch)
Stored 'newsgroups_test' (Bunch)
Stored 'X_train' (list)
Stored 'X_test' (list)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'X_train_tfidf' (csr_matrix)
Stored 'X_test_tfidf' (csr_matrix)
Stored 'X_train_pca' (ndarray)
Stored 'X_test_pca' (ndarray)
